In [2]:
import numpy as np
import pyvista as pv
import torch

In [75]:
etching = np.load('./bosch_data_1010_ratio08/bosch_sf_step_6_sf.npy')

In [18]:
etching = np.load('./bosch_data_1007_ratio08/bosch_sf_step_40_c4f8.npy')

In [48]:
etching = np.load('./bosch_data_1009_ratio08/bosch_sf_step_5_Ar.npy')

In [62]:
etching = np.load('./bosch_data_1009_ratio08/bosch_sf_step_1_c4f8.npy')

In [52]:
etching = np.load('./bosch_data_1009_ratio08/bosch_sf_step_5_sf.npy')

In [2]:
etching = np.load('./bosch_data_1007_ratio08/bosch_mask.npy')

In [6]:
from surface_normalize_bosch import surface_normal

center_with_direction=np.array([[50,50,140]])
range3D=np.array([[0, 100, 0, 100, 0, 800]])
InOrOut=[1]
yield_hist=np.array([None])
celllength=1e-5

maskTop=40
maskBottom=98
maskStep=10
maskCenter=[50, 50]
surface =  surface_normal(center_with_direction, range3D, InOrOut,celllength, yield_hist,\
                          maskTop, maskBottom, maskStep, maskCenter)

In [37]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 0

In [ ]:
(1.1**40)*100

In [ ]:
cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
# cyan = torch.Tensor(etching[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))

helf_plane = int(planes.shape[0]/2)

point_cloud = pv.PolyData(planes[:helf_plane, 3:])
vectors = planes[:helf_plane, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
print(vectors.shape)

In [ ]:
print(vectors[:10])

In [ ]:
print(np.linalg.norm(vectors[:10],axis=-1))

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))
planes[:10, 3:]

In [8]:
maskWall_indice = np.logical_and(planes[:, 5] > -102, planes[:, 5] < -158)

In [ ]:
maskWall_indice.shape

In [ ]:
test = planes[maskWall_indice, 3:]

print(test.shape)

In [ ]:
planes.shape

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))
maskWall_indice = np.logical_and(planes[:, 5] > 102, planes[:, 5] < 158)
test = planes[maskWall_indice, 3:]

test[:, 0] -= 50
test[:, 1] -= 50

np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1] )

vector_z = np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1])/10

print(vector_z)

new_vector = np.array([-test[:, 0], -test[:, 1], -vector_z]).T
new_vector_norm = np.linalg.norm(new_vector, axis=-1)

new_vector[:, 0] = np.divide(new_vector[:, 0], new_vector_norm)
new_vector[:, 1] = np.divide(new_vector[:, 1], new_vector_norm)
new_vector[:, 2] = np.divide(new_vector[:, 2], new_vector_norm)

planes[maskWall_indice, :3] = new_vector
print(test)

In [ ]:
test.shape

In [ ]:
planes.shape

In [ ]:
np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1] )

vector_z = np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1])/10

print(vector_z)

new_vector = np.array([-test[:, 0], -test[:, 1], -vector_z]).T
new_vector_norm = np.linalg.norm(new_vector, axis=-1)

new_vector[:, 0] = np.divide(new_vector[:, 0], new_vector_norm)
new_vector[:, 1] = np.divide(new_vector[:, 1], new_vector_norm)
new_vector[:, 2] = np.divide(new_vector[:, 2], new_vector_norm)


In [ ]:
helf_plane = int(planes.shape[0])
point_cloud = pv.PolyData(planes[:helf_plane, 3:])
vectors = planes[:helf_plane, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=3,
)

# Display the arrowscyan

plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))
helf_plane = int(planes.shape[0]/2)
point_cloud = pv.PolyData(planes[:helf_plane, 3:])
vectors = planes[:helf_plane, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=3,
)

# Display the arrowscyan

plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:, 50:, :,1]!=0, etching[:, 50:, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, 50:, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, 50:, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching[50, 50, 65:85]

In [38]:
etching = np.load('./bosch_data_1002_ratio08/bosch_sf_step_5_c4f8.npy')

In [ ]:
depo1 = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.sum(etching[:, :, :, :], axis=-1)==0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

# mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
# mask = mask.indices().numpy().T

# maskmesh = pv.PolyData(mask)
# maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
# maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.sum(etching[:, :, bottom-250:bottom, :], axis=-1)==0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-250:bottom,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, :, bottom-250:bottom,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, bottom-250:bottom, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-250:bottom,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, :, bottom-250:bottom,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-250:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-250:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-250:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-100:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-100:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-100:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()